<a href="https://colab.research.google.com/github/Idaliya19/Fin-markets/blob/Idaliia/Idaliia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ШАГ 1: ПОДКЛЮЧАЕМ БИБЛИОТЕКИ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import os

print("✓ Библиотеки подключены")

✓ Библиотеки подключены


In [3]:
# ШАГ 2: НАСТРОЙКИ

TICKER = "NVDA"  # Измени на "AAPL", "TSLA" и т.д.

PROJECT_PATH = "/content/nvda_agent" # Где сохранять файлы?

YEARS = 5 # Сколько лет данных брать?

print(f"Анализируем компанию: {TICKER}")
print(f"Папка проекта: {PROJECT_PATH}")
print(f"Период: последние {YEARS} лет\n")

Анализируем компанию: NVDA
Папка проекта: /content/nvda_agent
Период: последние 5 лет



In [4]:
# ШАГ 3: СОЗДАЕМ ПАПКИ ДЛЯ ФАЙЛОВ


folders = ["data_raw", "data_clean", "reports"] # Список нужных папок

for folder in folders:                          # Создаем каждую папку
    full_path = f"{PROJECT_PATH}/{folder}"
    os.makedirs(full_path, exist_ok=True)
    print(f"Папка создана: {folder}")

print("\nСтруктура папок готова!\n")

Папка создана: data_raw
Папка создана: data_clean
Папка создана: reports

Структура папок готова!



In [5]:
# ШАГ 4: ЗАГРУЖАЕМ ДАННЫЕ ИЗ ИНТЕРНЕТА

ticker = yf.Ticker(TICKER) # Подключаемся к Yahoo Finance

income_raw = ticker.income_stmt       # Отчет о прибылях и убытках
balance_raw = ticker.balance_sheet    # Баланс компании
cashflow_raw = ticker.cashflow        # Движение денег

print("Данные загружены!\n")

Данные загружены!



In [6]:
# ШАГ 5: СОХРАНЯЕМ СЫРЫЕ ДАННЫЕ

income_raw.to_csv(f"{PROJECT_PATH}/data_raw/income_stmt_raw.csv")
balance_raw.to_csv(f"{PROJECT_PATH}/data_raw/balance_sheet_raw.csv")
cashflow_raw.to_csv(f"{PROJECT_PATH}/data_raw/cashflow_raw.csv")


In [7]:
# ФУНКЦИЯ: ВЗЯТЬ ПОСЛЕДНИЕ N ЛЕТ

def take_recent_years(df, n=YEARS):

    cols_dt = pd.to_datetime(df.columns, errors='coerce')  # Превращаем названия колонок в даты

    mapping = dict(zip(df.columns, cols_dt))  # Связываем каждую колонку с ее датой

    sorted_cols = sorted(df.columns, key=lambda c: mapping[c], reverse=True)   # Сортируем колонки от новых к старым

    take = sorted_cols[:n]     # Берем первые n колонок (самые свежие)

    take_sorted = sorted(take, key=lambda c: mapping[c], reverse=True)     # Сортируем их еще раз по дате (от новых к старым)

    return df.loc[:, take_sorted].copy()     # Возвращаем только эти колонки

In [8]:
# ШАГ 6: ОБРАБАТЫВАЕМ ДАННЫЕ (income statement)

income_5y = take_recent_years(income_raw, YEARS) # Берем последние 5 лет

income_5y.index = (
    income_5y.index
    .str.strip()            # убираем лишние пробелы по краям
    .str.lower()            # делаем строчные буквы
    .str.replace(r"\s+", "_", regex=True)  # заменяем пробелы на подчёркивания
)

keep_income = [
    "total_revenue",             # Общая выручка
    "operating_revenue",         # Операционная выручка
    "cost_of_revenue",           # Себестоимость
    "gross_profit",              # Валовая прибыль
    "operating_income",          # Операционная прибыль
    "ebit",                      # Прибыль до вычета процентов и налогов
    "ebitda",                    # EBITDA
    "net_income",                # Чистая прибыль
    "research_and_development",               # Затраты на R&D
    "selling_general_and_administration",     # Административные расходы
    "operating_expense",                      # Операционные расходы
    "total_expenses",                         # Все расходы
    "interest_expense",           # Процентные расходы
    "interest_income",            # Процентные доходы
    "other_income_expense",       # Прочие доходы/расходы
    "pretax_income",              # Прибыль до налогов
    "tax_provision",              # Налоги
    "basic_eps",                  # Базовая прибыль на акцию
    "diluted_eps",                # Разводненная прибыль на акцию
    "basic_average_shares",       # Среднее число акций (базовое)
    "diluted_average_shares"      # Среднее число акций (разводнённое)
]

income_clean = income_5y.loc[income_5y.index.isin(keep_income)].copy()

income_clean

,2025-01-31,2024-01-31,2023-01-31,2022-01-31
ebitda,8.613700e+10,3.558300e+10,5.986000e+09,1.135100e+10
ebit,8.427300e+10,3.407500e+10,4.443000e+09,1.017700e+10
interest_expense,2.470000e+08,2.570000e+08,2.620000e+08,2.360000e+08
interest_income,1.786000e+09,8.660000e+08,2.670000e+08,2.900000e+07
total_expenses,4.904400e+10,2.795000e+10,2.139700e+10,1.687300e+10
diluted_average_shares,2.480400e+10,2.494000e+10,2.507000e+10,2.535000e+10
basic_average_shares,2.455500e+10,2.469000e+10,2.487000e+10,2.496000e+10
diluted_eps,2.940000e+00,1.190000e+00,1.740000e-01,3.850000e-01
basic_eps,2.970000e+00,1.210000e+00,1.760000e-01,3.910000e-01
net_income,7.288000e+10,2.976000e+10,4.368000e+09,9.752000e+09


In [9]:
# ШАГ 6: ОБРАБАТЫВАЕМ ДАННЫЕ (Balance sheet)

balance_5y = take_recent_years(balance_raw, YEARS)

balance_5y.index = (
    balance_5y.index
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", "_", regex=True)
)

keep_balance = [
    "total_assets",
    "total_liabilities_net_minority_interest",
    "common_stock_equity", "stockholders_equity",
    "current_assets",
    "current_liabilities",
    "inventory",
    "accounts_receivable",
    "accounts_payable",
    "cash_and_cash_equivalents",
    "cash_cash_equivalents_and_short_term_investments",
    "net_debt",
    "total_debt",
    "long_term_debt",
    "current_debt",
    "retained_earnings",
    "net_ppe",
    "invested_capital",
    "working_capital"
]


balance_clean = balance_5y.loc[balance_5y.index.isin(keep_balance)].copy()

balance_clean

,2025-01-31,2024-01-31,2023-01-31,2022-01-31,2021-01-31
net_debt,NaN,2.429000e+09,7.564000e+09,8.956000e+09,6.116000e+09
total_debt,1.027000e+10,1.105600e+10,1.203100e+10,1.183100e+10,NaN
invested_capital,8.779000e+10,5.268700e+10,3.305400e+10,3.755800e+10,NaN
working_capital,6.207900e+10,3.371400e+10,1.651000e+10,2.449400e+10,NaN
common_stock_equity,7.932700e+10,4.297800e+10,2.210100e+10,2.661200e+10,NaN
stockholders_equity,7.932700e+10,4.297800e+10,2.210100e+10,2.661200e+10,NaN
retained_earnings,6.803800e+10,2.981700e+10,1.017100e+10,1.623500e+10,NaN
total_liabilities_net_minority_interest,3.227400e+10,2.275000e+10,1.908100e+10,1.757500e+10,NaN
long_term_debt,8.463000e+09,8.459000e+09,9.703000e+09,1.094600e+10,NaN
current_liabilities,1.804700e+10,1.063100e+10,6.563000e+09,4.335000e+09,NaN


In [10]:
# ШАГ 6: ОБРАБАТЫВАЕМ ДАННЫЕ (Cash flow)

cashflow_5y = take_recent_years(cashflow_raw, YEARS)

cashflow_5y.index = (
    cashflow_5y.index
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", "_", regex=True)
)

keep_cash = [
    "operating_cash_flow",
    "cash_flow_from_continuing_operating_activities",
    "free_cash_flow",
    "capital_expenditure",
    "purchase_of_ppe",
    "capital_expenditure_reported",
    "depreciation",
    "depreciation_and_amortization",
    "depreciation_amortization_depletion",
    "investing_cash_flow",
    "financing_cash_flow",
    "cash_dividends_paid",
    "common_stock_dividend_paid",
    "repayment_of_debt",
    "issuance_of_debt",
    "long_term_debt_payments",
    "long_term_debt_issuance"
]


cash_clean = cashflow_5y.loc[cashflow_5y.index.isin(keep_cash)].copy()

cash_clean

,2025-01-31,2024-01-31,2023-01-31,2022-01-31,2021-01-31
free_cash_flow,6.085300e+10,2.702100e+10,3.808000e+09,8.132000e+09,NaN
repayment_of_debt,-1.250000e+09,-1.250000e+09,0.000000e+00,-1.000000e+09,NaN
issuance_of_debt,NaN,0.000000e+00,0.000000e+00,4.977000e+09,4.968000e+09
capital_expenditure,-3.236000e+09,-1.069000e+09,-1.833000e+09,-9.760000e+08,NaN
financing_cash_flow,-4.235900e+10,-1.363300e+10,-1.161700e+10,1.865000e+09,NaN
cash_dividends_paid,-8.340000e+08,-3.950000e+08,-3.980000e+08,-3.990000e+08,NaN
common_stock_dividend_paid,-8.340000e+08,-3.950000e+08,-3.980000e+08,-3.990000e+08,NaN
long_term_debt_payments,-1.250000e+09,-1.250000e+09,0.000000e+00,-1.000000e+09,NaN
long_term_debt_issuance,NaN,0.000000e+00,0.000000e+00,4.977000e+09,4.968000e+09
investing_cash_flow,-2.042100e+10,-1.056600e+10,7.375000e+09,-9.830000e+09,NaN


In [11]:
# ШАГ 7: СОХРАНЯЕМ ОЧИЩЕННЫЕ ДАННЫЕ

income_clean.to_csv(f"{PROJECT_PATH}/data_clean/income_clean.csv")
balance_clean.to_csv(f"{PROJECT_PATH}/data_clean/balance_sheet_clean.csv")
cash_clean.to_csv(f"{PROJECT_PATH}/data_clean/cashflow_clean.csv")

In [12]:
income_clean

,2025-01-31,2024-01-31,2023-01-31,2022-01-31
ebitda,8.613700e+10,3.558300e+10,5.986000e+09,1.135100e+10
ebit,8.427300e+10,3.407500e+10,4.443000e+09,1.017700e+10
interest_expense,2.470000e+08,2.570000e+08,2.620000e+08,2.360000e+08
interest_income,1.786000e+09,8.660000e+08,2.670000e+08,2.900000e+07
total_expenses,4.904400e+10,2.795000e+10,2.139700e+10,1.687300e+10
diluted_average_shares,2.480400e+10,2.494000e+10,2.507000e+10,2.535000e+10
basic_average_shares,2.455500e+10,2.469000e+10,2.487000e+10,2.496000e+10
diluted_eps,2.940000e+00,1.190000e+00,1.740000e-01,3.850000e-01
basic_eps,2.970000e+00,1.210000e+00,1.760000e-01,3.910000e-01
net_income,7.288000e+10,2.976000e+10,4.368000e+09,9.752000e+09


In [13]:
balance_clean

,2025-01-31,2024-01-31,2023-01-31,2022-01-31,2021-01-31
net_debt,NaN,2.429000e+09,7.564000e+09,8.956000e+09,6.116000e+09
total_debt,1.027000e+10,1.105600e+10,1.203100e+10,1.183100e+10,NaN
invested_capital,8.779000e+10,5.268700e+10,3.305400e+10,3.755800e+10,NaN
working_capital,6.207900e+10,3.371400e+10,1.651000e+10,2.449400e+10,NaN
common_stock_equity,7.932700e+10,4.297800e+10,2.210100e+10,2.661200e+10,NaN
stockholders_equity,7.932700e+10,4.297800e+10,2.210100e+10,2.661200e+10,NaN
retained_earnings,6.803800e+10,2.981700e+10,1.017100e+10,1.623500e+10,NaN
total_liabilities_net_minority_interest,3.227400e+10,2.275000e+10,1.908100e+10,1.757500e+10,NaN
long_term_debt,8.463000e+09,8.459000e+09,9.703000e+09,1.094600e+10,NaN
current_liabilities,1.804700e+10,1.063100e+10,6.563000e+09,4.335000e+09,NaN


In [14]:
cash_clean

,2025-01-31,2024-01-31,2023-01-31,2022-01-31,2021-01-31
free_cash_flow,6.085300e+10,2.702100e+10,3.808000e+09,8.132000e+09,NaN
repayment_of_debt,-1.250000e+09,-1.250000e+09,0.000000e+00,-1.000000e+09,NaN
issuance_of_debt,NaN,0.000000e+00,0.000000e+00,4.977000e+09,4.968000e+09
capital_expenditure,-3.236000e+09,-1.069000e+09,-1.833000e+09,-9.760000e+08,NaN
financing_cash_flow,-4.235900e+10,-1.363300e+10,-1.161700e+10,1.865000e+09,NaN
cash_dividends_paid,-8.340000e+08,-3.950000e+08,-3.980000e+08,-3.990000e+08,NaN
common_stock_dividend_paid,-8.340000e+08,-3.950000e+08,-3.980000e+08,-3.990000e+08,NaN
long_term_debt_payments,-1.250000e+09,-1.250000e+09,0.000000e+00,-1.000000e+09,NaN
long_term_debt_issuance,NaN,0.000000e+00,0.000000e+00,4.977000e+09,4.968000e+09
investing_cash_flow,-2.042100e+10,-1.056600e+10,7.375000e+09,-9.830000e+09,NaN


In [20]:
# market value ratios
price_per_share = ticker.info['currentPrice']
latest_period = income_clean.columns[0]

In [19]:
revenue = income_clean.loc['total_revenue', latest_period]
ebitda = income_clean.loc['ebitda', latest_period]
ebit = income_clean.loc['ebit', latest_period]
net_income = income_clean.loc['net_income', latest_period]
diluted_shares = income_clean.loc['diluted_average_shares', latest_period]
stockholders_equity = balance_clean.loc['stockholders_equity', latest_period]
total_debt = balance_clean.loc['total_debt', latest_period]
cash = balance_clean.loc['cash_and_cash_equivalents', latest_period]
operating_cash_flow = cash_clean.loc['operating_cash_flow', latest_period]
free_cash_flow = cash_clean.loc['free_cash_flow', latest_period]

In [21]:
market_cap = price_per_share * diluted_shares
enterprise_value = market_cap + total_debt - cash

In [22]:
pe_ratio = market_cap / net_income if net_income != 0 else None
pb_ratio = market_cap / stockholders_equity if stockholders_equity != 0 else None
ps_ratio = market_cap / revenue if revenue != 0 else None

In [24]:
previous_period = income_clean.columns[1]
net_income_prev = income_clean.loc['net_income', previous_period]
earnings_growth = ((net_income - net_income_prev) / net_income_prev * 100) if net_income_prev != 0 else None
peg_ratio = pe_ratio / earnings_growth if earnings_growth and earnings_growth != 0 else None

In [25]:
ev_ebit = enterprise_value / ebit if ebit != 0 else None
ev_ebitda = enterprise_value / ebitda if ebitda != 0 else None
ev_revenue = enterprise_value / revenue if revenue != 0 else None
p_ocf = market_cap / operating_cash_flow if operating_cash_flow != 0 else None
p_fcf = market_cap / free_cash_flow if free_cash_flow != 0 else None

In [27]:
market_ratios = pd.DataFrame({
    'Название': ['P/E', 'P/B', 'P/S', 'PEG', 'EV/EBIT', 'EV/EBITDA', 'EV/Revenue', 'P/OCF', 'P/FCF'],
    'Значение': [pe_ratio, pb_ratio, ps_ratio, peg_ratio, ev_ebit, ev_ebitda, ev_revenue, p_ocf, p_fcf]
})

market_ratios['Значение'] = market_ratios['Значение'].round(2)

In [28]:
market_ratios

,Название,Значение
0,P/E,61.48
1,P/B,56.48
2,P/S,34.33
3,PEG,0.42
4,EV/EBIT,53.19
5,EV/EBITDA,52.04
6,EV/Revenue,34.35
7,P/OCF,69.91
8,P/FCF,73.63


In [29]:
sector = ticker.info.get('sector', 'Unknown')
industry = ticker.info.get('industry', 'Unknown')

In [38]:
industry_peers = ['NVDA', 'AMD', 'INTC', 'AVGO', 'QCOM']
sector_peers = ['NVDA', 'AMD', 'INTC', 'AVGO', 'QCOM', 'TSM', 'MU', 'TXN', 'AMAT', 'LRCX', 'KLAC', 'ADI']

In [39]:
def get_peer_ratios(peer_list, group_name):
    all_ratios = {'pe': [], 'pb': [], 'ps': [], 'peg': [], 'ev_ebitda': []}
    yahoo_fields = {
        'pe': 'trailingPE',
        'pb': 'priceToBook',
        'ps': 'priceToSalesTrailing12Months',
        'peg': 'pegRatio',
        'ev_ebitda': 'enterpriseToEbitda'
    }
    for ticker_name in peer_list:
        try:
            info = yf.Ticker(ticker_name).info
            for ratio_name, yahoo_field in yahoo_fields.items():
                value = info.get(yahoo_field)
                if value:
                    all_ratios[ratio_name].append(value)
            print(f"  ✓ {ticker_name}")
        except:
            print(f"  ✗ {ticker_name}: ошибка")

    averages = {name: np.mean(values) if values else None
                for name, values in all_ratios.items()}
    return averages

industry_avg = get_peer_ratios(industry_peers, f"индустрии '{industry}'")
sector_avg = get_peer_ratios(sector_peers, f"сектору '{sector}'")


  ✓ NVDA
  ✓ AMD
  ✓ INTC
  ✓ AVGO
  ✓ QCOM
✓ Загружено 5 компаний

  ✓ NVDA
  ✓ AMD
  ✓ INTC
  ✓ AVGO
  ✓ QCOM
  ✓ TSM
  ✓ MU
  ✓ TXN
  ✓ AMAT
  ✓ LRCX
  ✓ KLAC
  ✓ ADI
✓ Загружено 12 компаний



In [42]:
market_ratios = pd.DataFrame({
    'Ratios': ['P/E', 'P/B', 'P/S', 'PEG', 'EV/EBIT', 'EV/EBITDA', 'EV/Revenue', 'P/OCF', 'P/FCF'],
    f'{TICKER}': [pe_ratio, pb_ratio, ps_ratio, peg_ratio, ev_ebit, ev_ebitda, ev_revenue, p_ocf, p_fcf],
    'Industry avg': [industry_avg['pe'], industry_avg['pb'], industry_avg['ps'], industry_avg['peg'], None, industry_avg['ev_ebitda'], None, None, None],
    'Sector avg': [sector_avg['pe'], sector_avg['pb'], sector_avg['ps'], sector_avg['peg'], None, sector_avg['ev_ebitda'], None, None, None]
})

In [43]:
market_ratios

,Ratios,NVDA,Industry avg,Sector avg
0,P/E,61.479069,166.817279,88.789331
1,P/B,56.482592,15.339697,12.402998
2,P/S,34.334847,14.271330,10.275069
3,PEG,0.424308,NaN,NaN
4,EV/EBIT,53.187564,NaN,NaN
5,EV/EBITDA,52.036588,28.438200,23.253167
6,EV/Revenue,34.347729,NaN,NaN
7,P/OCF,69.912069,NaN,NaN
8,P/FCF,73.629806,NaN,NaN
